# GONZAG cloud example

In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
from gonzag.config import *
from gonzag.io import *
from gonzag.utils import *
from gonzag.mod2sat import *
from gonzag.bilin_mapping import *
from gonzag.spectralysis import *


## Data

In [3]:
import xarray as xr
osn_url = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/swot_adac/eNATL60_surface_region_1'
dsmod = xr.open_zarr(osn_url, consolidated=True)

In [4]:
dsmod

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 611, y: 763)
Coordinates:
    depth          (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lat            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables:
    fmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosstsst       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>

In [5]:
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
l_griddist = False

In [6]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat["al"].to_dask()

In [7]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185024 ... 2...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    creator_name:              CMEMS - Sea Level Thematic Assembly Center
    creator_url:               http://marine.copernicus.eu
    history:                   2019-02-20T19:39:52Z: Creation
    institution:               CLS, CNES
    keywords:                  Oceans > Ocean Topography > Sea Surface Height
    keywords_vocabulary:       NetCDF COARDS Climate and Forecast Standard Names
    license:                   http://marine.copernicus.eu/web/27-service-com...
    platform:                  Altika
    processing_level:          L3
    product_version:           2019
    project:                   COPERNICUS MARINE ENVIRONMENT MONITORING SERVI...
    references:                http://marine.copernicus.eu
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [8]:
name_ssh_sat='sla_unfiltered'

## Time overlap between model and satellite ?
Assume that model is less than 1 year, satellite has at least one whole year, we take the overlapping period without considering which year is this

In [9]:
    (date1,date2), (Nts,Ntm) = GetTimeOverlap( dssat , dsmod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time"
   => first and last date:  2013-03-14T05:44:50.445185024 -- 2015-03-31T22:41:36.909480960
 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time_counter"
   => first and last date:  2010-02-01T00:30:00.000000000 -- 2010-04-30T23:30:00.000000000

 *** Earliest/latest dates:
   => for satellite data: 2013-03-14T05:44:50.445185024 2015-03-31T22:41:36.909480960 
   => for model     data: 2010-02-01T00:30:00.000000000 2010-04-30T23:30:00.000000000 

 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-02-01 -- 2014-04-30


## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [10]:
  
    ModelGrid = ModGrid( dsmod, date1 , date2 , dsmod, name_lsm_mod, distorded_grid=False )


 *** [GetTimeVector()] reading "time_counter" in dataset ...
   => 2136 records in TOTAL!

 *** [GetModelCoor()] Read model latitude (variable is "lat", with 2 dimensions! (763, 611) 

 *** [GetModelCoor()] Read model longitude (variable is "lon", with 2 dimensions! (763, 611) 


 *** what we use to define model land-sea mask:
    => "tmask" in dataset 

 *** [GridResolution()] Based on the longitude array, the model resolution ~=  0.016657295  degrees 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (763, 611)
     * horizontal resolution:  0.016657295  degrees or  1.8507920147292316  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  281.99 292.17
     * lat_min, lat_max =  30.01 40.01
     * should we pay attention to possible STRONG local distorsion in the grid:  Fa

In [11]:
SatelliteTrack = SatTrack( dssat, date1, date2, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...
 *** [GetTimeVector()] reading "time" in dataset ...
   => 4150072 records read...

 *** [GetSatCoor()] reading "latitude" in dataset ...
   => 4150072 records read...

 *** [GetSatCoor()] reading "longitude" in dataset ...
   => 4150072 records read...


 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  10236
       ==> time record indices: 14883510 to 19033581, included

       separated in 87 tracks


In [42]:
SatelliteTrack.time

<xarray.DataArray 'time' (time: 10236)>
array(['2014-02-02T23:14:13.207443968', '2014-02-02T23:14:14.187443968',
       '2014-02-02T23:14:15.167443968', ..., '2014-04-29T23:14:02.891097088',
       '2014-04-29T23:14:03.871097088', '2014-04-29T23:14:04.851097088'],
      dtype='datetime64[ns]')
Coordinates:
    latitude   (time) float64 dask.array<chunksize=(10236,), meta=np.ndarray>
    longitude  (time) float64 dask.array<chunksize=(10236,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2014-02-02T23:14:13.207443968 ... 2014-0...
Attributes:
    axis:           T
    long_name:      Time of measurement
    standard_name:  time

In [12]:
Solution0 = Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, 25 )


 *** "found" distance criterion when searching for nearest point on model grid is  1.3880940110469238  km

 *** Will use zoom boxes of width of 271 points for 1st attempts of nearest-point location...


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3880940110469238 135 )
      +++ Treated point: 16/162 
          ==> Sat. coordinates:     30.897 289.597
          ==> Model nearest point:  30.902 289.604  ( 62 456 )
      +++ Treated point: 32/162 
          ==> Sat. coordinates:     31.822 289.341
          ==> Model nearest point:  31.817 289.349  ( 128 441 )
      +++ Treated point: 48/162 
          ==> Sat. coordinates:     32.747 289.082
          ==> Model nearest point:  32.745 289.076  ( 196 425 )
      +++ Treated point: 64/162 
          ==> Sat. coordinates:     33.671 288.818
          ==> Model nearest point:  33.673 288.818  ( 265 410 )
      +++ Treated point: 80/162 
          ==> Sat. coordinates:     35.057 288.415
          ==> Mode

In [14]:

    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]
    iw = SaveTimeSeries( Solution0.time, \
                         nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
                                     Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] ), \
                         vvar, 'result0.nc', time_units='',\
                         vunits=vunits, vlnm=vlongN, missing_val=rmissval )


 *** About to write file "result0.nc"...


ValueError: failed to prevent overwriting existing key units in attrs on variable 'time'. This is probably an encoding field used by xarray to describe how a variable is serialized. To proceed, remove this key from the variable's attributes manually.

In [15]:
xd=nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
                                     Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] )

In [16]:
ivt=Solution0.time

In [17]:

    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]


In [18]:
(Nf,Nt) = xd.shape

In [19]:
print(Nf,Nt)

6 162


In [20]:
print(len(ivt))

162


In [21]:
print(len(vvar))

6


In [22]:
ncfile='test-result0.nc'
time_units=''
vlnm=vlongN
missing_val=rmissval

In [44]:
ivt.values

array(['2014-03-26T10:24:54.983470080', '2014-03-26T10:24:55.963470080',
       '2014-03-26T10:24:56.943470080', '2014-03-26T10:24:57.923470080',
       '2014-03-26T10:24:58.903470080', '2014-03-26T10:24:59.883470080',
       '2014-03-26T10:25:00.863470080', '2014-03-26T10:25:01.843470080',
       '2014-03-26T10:25:02.823470080', '2014-03-26T10:25:03.803470080',
       '2014-03-26T10:25:04.783470080', '2014-03-26T10:25:05.763470080',
       '2014-03-26T10:25:06.743470080', '2014-03-26T10:25:07.723470080',
       '2014-03-26T10:25:08.703470080', '2014-03-26T10:25:09.683470080',
       '2014-03-26T10:25:10.663470080', '2014-03-26T10:25:11.643470080',
       '2014-03-26T10:25:12.623470080', '2014-03-26T10:25:13.603470080',
       '2014-03-26T10:25:14.583470080', '2014-03-26T10:25:15.563470080',
       '2014-03-26T10:25:16.543470080', '2014-03-26T10:25:17.523470080',
       '2014-03-26T10:25:18.503470080', '2014-03-26T10:25:19.483470080',
       '2014-03-26T10:25:20.463470080', '2014-03-26

In [45]:
      foo0=xr.DataArray(xd[0],dims=['time'],coords=[ivt.astype(nmp.float64)])
      foo0.name=vvar[0]
      foo0.attrs["units"] = vunits[0]
      foo0.attrs["long_name"] = vlnm[0]
      ds=foo0.to_dataset(name = vvar[0])


In [46]:
ds

<xarray.Dataset>
Dimensions:   (time: 162)
Coordinates:
  * time      (time) float64 1.396e+18 1.396e+18 ... 1.396e+18 1.396e+18
Data variables:
    latitude  (time) float64 30.03 30.09 30.15 30.2 ... 39.78 39.84 39.9 39.95

In [47]:
      for jf in range(1,Nf):
        foo=xr.DataArray(xd[jf],dims=['time'],coords=[ivt.astype(nmp.float64)])
        foo.name=vvar[jf]
        foo.attrs["units"] = vunits[jf]
        foo.attrs["long_name"] = vlnm[jf]
        ds[vvar[jf]]=foo


In [48]:
ds

<xarray.Dataset>
Dimensions:         (time: 162)
Coordinates:
  * time            (time) float64 1.396e+18 1.396e+18 ... 1.396e+18 1.396e+18
Data variables:
    latitude        (time) float64 30.03 30.09 30.15 30.2 ... 39.84 39.9 39.95
    longitude       (time) float64 289.8 289.8 289.8 289.8 ... 286.9 286.9 286.9
    sossheig_np     (time) float64 0.439 0.4235 0.4411 ... -0.2922 0.4299 0.4298
    sossheig_bl     (time) float64 0.439 0.4245 0.4415 ... -0.2925 0.4299 0.4298
    sla_unfiltered  (time) float64 0.038 0.04 0.045 ... -0.061 -0.132 -0.116
    distance        (time) float64 1.123e+03 1.13e+03 ... 2.212e+03 2.219e+03

In [49]:
footime=xr.DataArray(ivt, dims=['time'],coords=[ivt.astype(nmp.float64)])


In [50]:
      ds['time_counter']=footime
 

In [51]:
ds

<xarray.Dataset>
Dimensions:         (time: 162)
Coordinates:
  * time            (time) float64 1.396e+18 1.396e+18 ... 1.396e+18 1.396e+18
Data variables:
    latitude        (time) float64 30.03 30.09 30.15 30.2 ... 39.84 39.9 39.95
    longitude       (time) float64 289.8 289.8 289.8 289.8 ... 286.9 286.9 286.9
    sossheig_np     (time) float64 0.439 0.4235 0.4411 ... -0.2922 0.4299 0.4298
    sossheig_bl     (time) float64 0.439 0.4245 0.4415 ... -0.2925 0.4299 0.4298
    sla_unfiltered  (time) float64 0.038 0.04 0.045 ... -0.061 -0.132 -0.116
    distance        (time) float64 1.123e+03 1.13e+03 ... 2.212e+03 2.219e+03
    time_counter    (time) datetime64[ns] 2014-03-26T10:24:54.983470080 ... 2...

In [52]:
ds.to_netcdf('test.nc','w', format='NETCDF4')